# ハイパラチューニング試し

In [1]:
import os
import networkx as nx
import torch
import numpy as np
import pandas as pd
import sys
sys.path.append("../codes")

from torch_geometric.data import Data, DataLoader
from torch_geometric.datasets import TUDataset

from model import GCN
from train_test import train, test

In [14]:
import torch.optim as optim

def get_optimizer(trial, model):
    # optimizer をAdamとMomentum SGDで探索
    optimizer_names = ['Adam', 'MomentumSGD', "Adagrad"]
    optimizer_name = trial.suggest_categorical('optimizer', optimizer_names)

    # weight decayの探索
    weight_decay = trial.suggest_loguniform('weight_decay', 1e-10, 1e-3)

    # optimizer_nameで分岐
    if optimizer_name == optimizer_names[0]: 
        adam_lr = trial.suggest_loguniform('adam_lr', 1e-5, 1e-1)
        optimizer = optim.Adam(model.parameters(), lr=adam_lr, weight_decay=weight_decay)
    elif optimizer_name == optimizer_names[2]:
        adam_lr = trial.suggest_loguniform('adam_lr', 1e-5, 1e-1)
        optimizer = optim.Adagrad(model.parameters(), lr=adam_lr, weight_decay=weight_decay)
    else:
        momentum_sgd_lr = trial.suggest_loguniform('momentum_sgd_lr', 1e-5, 1e-1)
        optimizer = optim.SGD(model.parameters(), lr=momentum_sgd_lr,
                            momentum=0.9, weight_decay=weight_decay)
    return optimizer


In [21]:
import optuna
import torch

EPOCH = 10

dataset = TUDataset(root='data/TUDataset', name="REDDIT-BINARY")
torch.manual_seed(12345)
dataset = dataset.shuffle()

index = int(len(dataset) * 0.7)
train_dataset = dataset[:index]
test_dataset = dataset[index:]

train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)

def objective(trial):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    model = GCN(hidden_channels=64, dataset=dataset).to(device)
    optimizer = get_optimizer(trial, model)
    criterion = torch.nn.CrossEntropyLoss()
    
    # Training
    for epoch in range(EPOCH):
        train(model, train_loader, criterion, optimizer, device, x="degree")
        
    # Evaluation
    accuracy = test(model, test_loader, device, x="degree")
  
    # 返り値が最小となるようにハイパーパラメータチューニングが実行される
    return 1.0 - accuracy

In [22]:
study = optuna.create_study()
study.optimize(objective, n_trials=100)

[I 2021-07-03 02:09:45,604] A new study created in memory with name: no-name-5a3b4b90-5935-403f-907c-bd65674f4068
[I 2021-07-03 02:10:32,405] Trial 0 finished with value: 0.5066666666666666 and parameters: {'optimizer': 'Adagrad', 'weight_decay': 5.373332673782192e-10, 'adam_lr': 1.4488705372885993e-05}. Best is trial 0 with value: 0.5066666666666666.
[I 2021-07-03 02:11:20,886] Trial 1 finished with value: 0.30166666666666664 and parameters: {'optimizer': 'Adam', 'weight_decay': 0.000305649020550488, 'adam_lr': 0.001489638204378908}. Best is trial 1 with value: 0.30166666666666664.
[I 2021-07-03 02:12:07,538] Trial 2 finished with value: 0.5066666666666666 and parameters: {'optimizer': 'MomentumSGD', 'weight_decay': 1.0382669808627678e-10, 'momentum_sgd_lr': 1.3315403671341518e-05}. Best is trial 1 with value: 0.30166666666666664.
[I 2021-07-03 02:12:53,533] Trial 3 finished with value: 0.3383333333333334 and parameters: {'optimizer': 'MomentumSGD', 'weight_decay': 5.6110494665700243e

In [7]:
study.best_params  # Get best parameters for the objective function.

{'optimizer': 'Adam',
 'weight_decay': 7.332069279877374e-10,
 'adam_lr': 0.017842267811527854}

In [24]:
1 - study.best_value  # Get best objective value.

0.755